In [1]:
import pandas as pd

In [2]:
def excel_to_sql_multi_sheet(file_path):
    # Read all sheets into a dictionary
    sheet_dict = pd.read_excel(file_path, sheet_name=None, engine='openpyxl')

    sql_output = ""

    # Type mapping
    sql_types = {
        'int64': 'INT',
        'float64': 'FLOAT',
        'object': 'VARCHAR(255)',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'DATETIME'
    }

    # Process each sheet
    for sheet_name, df in sheet_dict.items():
        table_name = sheet_name.replace(" ", "_")  # make table name safe
        create_cols = []
        for col in df.columns:
            dtype = str(df[col].dtype)
            sql_type = sql_types.get(dtype, 'VARCHAR(255)')
            create_cols.append(f"`{col}` {sql_type}")

        create_table = f"-- Sheet: {sheet_name}\nCREATE TABLE `{table_name}` (\n  " + ",\n  ".join(create_cols) + "\n);\n"
        insert_statements = []
        for _, row in df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append("NULL")
                elif isinstance(value, str):
                    escaped = value.replace("'", "\\'")
                    values.append(f"'{escaped}'")
                elif isinstance(value, (pd.Timestamp, bool)):
                    values.append(f"'{value}'")
                else:
                    values.append(str(value))
            insert = f"INSERT INTO `{table_name}` ({', '.join([f'`{col}`' for col in df.columns])}) VALUES ({', '.join(values)});"
            insert_statements.append(insert)

        sql_output += create_table + "\n" + "\n".join(insert_statements) + "\n\n"

    return sql_output

In [3]:
# Example usage
file_path = 'food_database.xlsx'
sql_code = excel_to_sql_multi_sheet(file_path)

# Save to output.sql
with open('output.sql', 'w', encoding='utf-8') as f:
    f.write(sql_code)

print("SQL code for all sheets saved to output.sql")

SQL code for all sheets saved to output.sql
